<a href="https://colab.research.google.com/github/aayushkubb/Deep_Learning_Tutorial/blob/master/Retrieving_Information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deep Learning for NLP
--

Retrieving Information
--

Information retrieval is one of the highly used applications of NLP and it is
quite tricky. The meaning of the words or sentences not only depends on the exact words used but also on the context and meaning. Two sentences may be of completely different words but can convey the same meaning. We should be able to capture that as well.

An information retrieval (IR) system allows users to efficiently search documents and retrieve meaningful information based on a search text/query.

<img src="https://drive.google.com/uc?id=1QOtLrYLecQSPH3D_SgaFg12urVyeOu3b" />

Problem
--
Information retrieval using word embeddings.

Solution
--
There are multiple ways to do Information retrieval. But we will see how to do it using word embeddings, which is very effective since it takes context also into consideration. 

We will just use the pretrained word2vec in this case. 
( Src : https://radimrehurek.com/gensim/models/word2vec.html )

Let’s take a simple example and see how to build a document retrieval using query input. Let’s say we have 4 documents in our database as below. (Just showcasing how it works. We will have too many documents in a real-world application.)

In [ ]:
Doc1 = ["With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders." ]
        
Doc2 = ["Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data."]

Doc3 = ["He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems."] 

Doc4 = ["But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg."]

Assume we have numerous documents like this. And you want to retrieve the most relevant one, for the query “cricket.” Let’s see how to build it.

**query = "cricket"**

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Step 7.1.1 : Import the libraries
import gensim
from gensim.models import Word2Vec
import numpy as np
import nltk
import itertools
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import scipy
from scipy import spatial
from nltk.tokenize.toktok import ToktokTokenizer
import re

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

In [ ]:
# Step 7.1.2 - Create/import documents

# Doc1 , Doc2 , Doc3 and Doc4 as defined above in the code.

# Put all the documents in one list
fin = Doc1+Doc2+Doc3+Doc4
print(fin)

['With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.', 'Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.', 'He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.', 'But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.']


In [ ]:
# import gensim package
import gensim

# load the saved model
model = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
print(model)
print(gensim.models.Word2Vec())

Word2Vec(vocab=0, size=100, alpha=0.025)


In [ ]:
# Step 7.1.4 : Create IR system

# Now we build the information retrieval system:

# Preprocessing
def remove_stopwords(text, is_lower_case=False):
 pattern = r'[^a-zA-z0-9\s]'
 text = re.sub(pattern, "" , text)
 

 tokens = tokenizer.tokenize(text)
 tokens = [token.strip() for token in tokens]
 if is_lower_case:
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  
 else:
  filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  
 return filtered_text


# Function to get the embedding vector for n dimension, we have used "300"
def get_embedding(word):
 if word in model.wv.vocab:
  return model[word]
 else:
  return np.zeros(300)

In [ ]:
# just to see the word vector for some word.
print(model['cricket'])
print("-----------------------------------")
print(len(model['cricket']))
print(np.mean(np.array(model['cricket']), axis=0))

# just to see no. of words in Doc1
print(len(tokenizer.tokenize(Doc1)))


# so, we would get 47 word vectors, each of size 300. 
# One vector for one word. 
# so, we could get the mean of each vector 
# so that we reduce of the values we have to handle  

[-3.67187500e-01 -1.21582031e-01  2.85156250e-01  8.15429688e-02
  3.19824219e-02 -3.19824219e-02  1.34765625e-01 -2.73437500e-01
  9.46044922e-03 -1.07421875e-01  2.48046875e-01 -6.05468750e-01
  5.02929688e-02  2.98828125e-01  9.57031250e-02  1.39648438e-01
 -5.41992188e-02  2.91015625e-01  2.85156250e-01  1.51367188e-01
 -2.89062500e-01 -3.46679688e-02  1.81884766e-02 -3.92578125e-01
  2.46093750e-01  2.51953125e-01 -9.86328125e-02  3.22265625e-01
  4.49218750e-01 -1.36718750e-01 -2.34375000e-01  4.12597656e-02
 -2.15820312e-01  1.69921875e-01  2.56347656e-02  1.50146484e-02
 -3.75976562e-02  6.95800781e-03  4.00390625e-01  2.09960938e-01
  1.17675781e-01 -4.19921875e-02  2.34375000e-01  2.03125000e-01
 -1.86523438e-01 -2.46093750e-01  3.12500000e-01 -2.59765625e-01
 -1.06933594e-01  1.04003906e-01 -1.79687500e-01  5.71289062e-02
 -7.41577148e-03 -5.59082031e-02  7.61718750e-02 -4.14062500e-01
 -3.65234375e-01 -3.35937500e-01 -1.54296875e-01 -2.39257812e-01
 -3.73046875e-01  2.27355

In [ ]:
# Getting average vector for each document
out_dict = {}
for sen in fin:  # this loop will pick one sentence at a time from fin (final document)
 average_vector = (np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(remove_stopwords(sen))]), axis=0))
 dict = { sen : (average_vector) }
 out_dict.update(dict)

# Function to calculate the similarity between the query vector and document vector
def get_sim(query_embedding, average_vector_doc):
 sim = [(1 - scipy.spatial.distance.cosine(query_embedding, average_vector_doc))]
 return sim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [ ]:
print(out_dict)

{'With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.': array([ 0.0059648 ,  0.07422152,  0.02200317,  0.05176891, -0.05416038,
       -0.11055964,  0.00973372, -0.01471086,  0.06308608,  0.03867409,
       -0.02027199,  0.00665144, -0.01004583,  0.03567227, -0.1430886 ,
        0.01957009,  0.08405928,  0.01300049,  0.00619533, -0.03672374,
        0.0830411 , -0.01095581, -0.0281979 , -0.01133589,  0.088304  ,
       -0.07389138, -0.0822199 ,  0.04952309,  0.03541704, -0.03258722,
       -0.05574174, -0.10178999,  0.03765314,  0.09196888,  0.02263849,
       -0.10769298,  0.0222057 , -0.05080899, -0.03323087,  0.06027499,
        0.00488836, -0.040003  ,  0.09283066, -0.0387158 , -0.09643277,
       -0.09111439, -0.01537531,  0.04396612,  0.00518799, -0.02616189,
       -0.05735085,  0.03955494,  0.0

In [ ]:
# Rank all the documents based on the similarity to get relevant docs
def Ranked_documents(query):
 query_words = (np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(query.lower())],dtype=float), axis=0))
 rank = []
 
 for k,v in out_dict.items():
  rank.append((k, get_sim(query_words, v)))
 
 rank = sorted(rank,key=lambda t: t[1], reverse=True)
 print('Ranked Documents :')
 return rank

In [ ]:
# Call the IR function with a query
Ranked_documents("cricket")

Ranked Documents :


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


[('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [0.44954328830341783]),
 ('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [0.23973446930269127]),
 ('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.',
  [0.1832371201201335]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
  [0.179950

In [ ]:
# Let’s take one more example as may be driving.
Ranked_documents("driving")


Ranked Documents :


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


[('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.',
  [0.3594728772380067]),
 ('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [0.19042557661139026]),
 ('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [0.1706653724240128]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
  [0.0887230

We can use the same approach and scale it up for as many documents as possible. 

For more accuracy, we can build our own embeddings, as we learned in Chapter 4, for specific industries since the one we are using is generalized.
(Although it is time consuming task)

This is the fundamental approach that can be used for many applications like the following:
1. Search engines (Internet Search Engines)
2. Document retrieval
3. Passage retrieval
4. Question and answer

<img src="https://drive.google.com/uc?id=1kGARF7WUBYrDKbRkQh5l7V01pDTYCwiU">



It’s been proven that results will be good when queries are longer and
the result length is shorter. That’s the reason we don’t get great results in search engines when the search query has lesser number of words.